In [ ]:
import requests
import pandas as pd
import re
from datetime import datetime
import json
import pysolr 
from tqdm import tqdm
CORE_NAME = 'reddit'
solr_url = f'http://34.125.52.100:8983/solr/'

In [45]:
def add_fields():
    data = {
        "add-field": [
                {
                    "name": "upvotes",
                    "type": "pint",
                    "indexed": True,
                    "multiValued": False
                },
            {
                    "name": "subreddit",
                    "type": "string",
                    "indexed": True,
                    "multiValued": False
                },
            {
                    "name": "full_link",
                    "type": "string",
                    "indexed": True,
                    "multiValued": False
                },
            {
                    "name": "title",
                    "type": "string",
                    "indexed": True,
                    "multiValued": False
                },
            {
                    "name": "selftext",
                    "type": "text_en",
                    "indexed": True,
                    "multiValued": False
                },
            {
                    "name": "author",
                    "type": "string",
                    "indexed": True,
                    "multiValued": False
                },
            {
                    "name": "is_submission",
                    "type": "boolean",
                    "indexed": True,
                    "multiValued": False
                },
            {
                    "name": "topic",
                    "type": "string",
                    "indexed": True,
                    "multiValued": False
                },
            {
                    "name": "created_at",
                    "type": "pdate",
                    "indexed": True,
                    "multiValued": False
                },
            {
                    "name": "body",
                    "type": "text_en",
                    "indexed": True,
                    "multiValued": False
                },
            {
                    "name": "parent_id",
                    "type": "string",
                    "indexed": True,
                    "multiValued": False
                },
            {
                    "name": "parent_body",
                    "type": "text_en",
                    "indexed": True,
                    "multiValued": False
                }
                
            ]
        }

    print(requests.post(solr_url + CORE_NAME + "/schema", json=data).json())

In [47]:
data = pd.read_csv('D:\\scrape2.csv')
data['created_at'] = data['created_utc'].apply(lambda x:datetime.utcfromtimestamp(x).strftime('%Y-%m-%dT%H:%M:%SZ') )
data= data[['id', 'subreddit', 'full_link', 'title', 'body', 'selftext',
       'author', 'is_submission', 'parent_id', 'parent_body', 'topic', 'created_at','upvotes']]

def toLower(x):
    if x == 'Foodforthought':
        return x.lower()
    else:
        return x
    
data['subreddit'] = data['subreddit'].apply(lambda x: toLower(x))
connection = pysolr.Solr(solr_url + CORE_NAME, always_commit=True, timeout=50000)

{'responseHeader': {'status': 0, 'QTime': 881}}


In [48]:
for i in tqdm(range(0,data.shape[0],100)):
    test =  data.iloc[i:i+100]
    result = test.to_json(orient="records")
    parsed = json.loads(result)
    json.dumps(parsed, indent=4)  
    connection.add(parsed)

100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:03<00:00,  2.90it/s]


In [2]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

In [24]:
# query ='do you think global warming is true?'
query ='what is ocean pollution?'
qembeddings = model.encode(query, convert_to_tensor=True)

result = requests.get('http://localhost:8983/solr/IRF22P1/select?debugQuery=false&defType=dismax&fl=id%2Cscore%2Cbody%2Cparent_body&indent=true&q.op=OR&q='+query+'&qf=parent_body&rows=20')
#result = requests.get('http://localhost:8983/solr/IRF22P1/select?indent=true&q.op=OR&q=parent_body%3A'+query+'&rows=100')
response = None
score = None
matchedBest =None
for doc in result.json()['response']['docs']:
    inp = doc['parent_body']
    inpemb= model.encode(inp, convert_to_tensor=True)
    cosine_score = util.cos_sim(inpemb, qembeddings)
    print(str(cosine_score) + ' '+str(doc['score']))
    if score == None or cosine_score > score:
        score = cosine_score
        matchedBest = inp
        response = doc['body']

print('query : '+query)
print('Just Solr match : '+ result.json()['response']['docs'][0]['parent_body'])
print('Just Solr response : '+ result.json()['response']['docs'][0]['body'])
print('Embedding match : '+ matchedBest)
print('Embedding Response : '+response)

tensor([[0.2922]]) 7.587557
tensor([[0.3559]]) 7.0285263
tensor([[0.5414]]) 6.883585
tensor([[0.6002]]) 6.883585
tensor([[0.5711]]) 6.883585
tensor([[0.3606]]) 6.1255436
tensor([[0.3606]]) 6.1255436
tensor([[0.3850]]) 5.67769
tensor([[0.3850]]) 5.67769
tensor([[0.3850]]) 5.67769
tensor([[0.3563]]) 5.31948
tensor([[0.1858]]) 5.216258
tensor([[0.3477]]) 5.055812
tensor([[0.3698]]) 5.0014653
tensor([[0.4222]]) 4.9639096
tensor([[0.4222]]) 4.9639096
tensor([[0.3392]]) 4.899925
tensor([[0.3392]]) 4.899925
tensor([[0.3392]]) 4.899925
tensor([[0.3392]]) 4.899925
query : what is ocean pollution?
Just Solr match : there is a certain idiocy in straws being singled out, but the commenter is lamenting the lack of plastic straws. Like basically his logic is: “if only companies weren’t polluting the ocean with so much plastic, then it would be more okay for us to pollute the ocean with specifically plastic straws because I really like plastic straws.”
Just Solr response : only the ocean was never po

In [9]:
result.json()

{'responseHeader': {'status': 400,
  'QTime': 38,
  'params': {'q': 'input:do you think global warming is true?',
   'indent': 'true',
   'q.op': 'OR',
   'rows': '100'}},
 'error': {'metadata': ['error-class',
   'org.apache.solr.common.SolrException',
   'root-error-class',
   'org.apache.solr.common.SolrException'],
  'msg': 'undefined field input',
  'code': 400}}

In [11]:
result = requests.get('http://localhost:8983/solr/IRF22P1/select?debugQuery=false&defType=dismax&fl=id%2Cscore%2Cbody%2Cparent_body&indent=true&q.op=OR&q='+query+'&qf=parent_body&rows=20')
result.json()

{'responseHeader': {'status': 0,
  'QTime': 3,
  'params': {'q': 'do you think global warming is true?',
   'defType': 'dismax',
   'indent': 'true',
   'qf': 'parent_body',
   'fl': 'id,score,body,parent_body',
   'q.op': 'OR',
   'rows': '20',
   'debugQuery': 'false'}},
 'response': {'numFound': 88415,
  'start': 0,
  'maxScore': 9.250551,
  'numFoundExact': True,
  'docs': [{'id': 'hlumwsu',
    'body': "Global warming is literally the biggest threat to human lives. I can't think of anything close other than all out nuclear war, but the difference is that the effects of global warming are inevitable.",
    'parent_body': 'Fair enough. What makes you think other sources are not biased (places that think global warming is the biggest threat ever.',
    'score': 9.250551},
   {'id': 'hcgxlbq',
    'body': 'Just google it. It doesn’t take a scientist to find it on line. Who are you to tell me the rules? Just who do you think you are?',
    'parent_body': "Look it up where? You make the

In [32]:
from sentence_transformers import SBertSummarizer

body = 'Text body that you want to summarize with BERT'
model = SBertSummarizer('paraphrase-MiniLM-L6-v2')
result = model(body, num_sentences=3)

ImportError: cannot import name 'SBertSummarizer' from 'sentence_transformers' (C:\Users\aniru\miniconda3\lib\site-packages\sentence_transformers\__init__.py)

In [29]:
from sentence_transformers import util
util.

[]

In [30]:
sentences

["\\&gt;Hell, there's more plastic than fish in the ocean now.\n\nGod I hate hyperbole masquerading as fact. The estimated amount of plastic in the ocean is 270,000 tonnes.\n\nThe estimated mass of all fishlife is 2 billion tonnes, meaning the total mass of plastic is about 0.01% of the mass of fishlife.\n\nYour response not only does not answer my question, but shows you lack a sense proportion and aren't qualified to answer it."]